In [1]:
import pandas as pd 
import numpy as np 
import os
from google.cloud import bigquery
from google.oauth2 import service_account
# declaramos las credenciales al entorno de desarollo
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "..\\..\\bd\\0_docs\\datahub-deacero-adojeda.json"

In [197]:
if True: 
    client = bigquery.Client()
    consulta = """
        SELECT 
        FORMAT_DATE('%Y-%m-%d', PARSE_DATE('%B.%Y', mes_anio)) AS MesAnio, # Generamos la fecha
        nom_ubicacion,
        nom_direccion,
        sum(toneladas_facturadas) as toneladas_facturadas
        FROM `datahub-deacero.mart_comercial.comercial` 
        where (mes_anio is not null)
        and (toneladas_facturadas is not null)
        GROUP BY
        MesAnio,
        nom_ubicacion,
        nom_direccion
            """
    query = client.query(consulta)
    pl = query.to_dataframe(create_bqstorage_client=True)

c:\Users\adojeda\OneDrive - deacero.com\dev\0_envs\.bigquery\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [1]:
kpi2 = pl.groupby("nom_ubicacion").agg({"MesAnio": "max"}).rename(columns = {"MesAnio":"fecha_fin"})
kpis = pl.groupby("nom_ubicacion").agg({"MesAnio": "min", "nom_direccion": "count", "toneladas_facturadas":"mean"}).rename(columns = {"MesAnio":"fecha_inicio", "nom_direccion": "mcc", "toneladas_facturadas": "ton_media"} )
kpis = kpis.merge(kpi2, left_index=True, right_index=True)
kpis["r_ton_med"] = pd.qcut(kpis.ton_media, q = 4)
kpis["r_mcc_med"] = pd.qcut(kpis.ton_media, q = 4)
kpis.fecha_inicio = pd.to_datetime(kpis.fecha_inicio) 
kpis.fecha_fin = pd.to_datetime(kpis.fecha_fin) 
kpis["qdate"] = kpis.fecha_inicio.dt.year.mul(100) + kpis.fecha_inicio.dt.quarter
kpis["qdate"]= np.where(kpis.fecha_inicio.dt.year < 2024, kpis.fecha_inicio.dt.year.mul(100), kpis["qdate"] )
kpis["msc"] = kpis.fecha_fin.max() - kpis.fecha_fin
kpis["msc"] = kpis["msc"].dt.days.div(30).round(0)
kpis["r_msc"] = pd.cut(kpis.msc, bins = [-np.inf, 1, 2, 3, 4, 8 , 10,12, np.inf])

def freq(x): 
    # pandas time series
    x = x[x.toneladas_facturadas > 0]
    if x.shape[0]< 2 :
        return None
    else:  
        x = round(x.MesAnio.diff().mean().days / 30) 
        return x
base = pl.reset_index().groupby(["MesAnio", "nom_ubicacion"]).toneladas_facturadas.sum().reset_index()
base.MesAnio = pd.to_datetime(base.MesAnio)
base_gb = base.groupby("nom_ubicacion")
calculo_frecuencia_media = base_gb.apply(lambda x :  freq(x))
calculo_frecuencia_media = calculo_frecuencia_media.reset_index().rename(columns = {0: "Freq"})
kpis = kpis.reset_index().merge(calculo_frecuencia_media, on = "nom_ubicacion")
kpis ["r_freq"] = pd.cut(x = kpis.Freq.fillna(24), bins = [0, 1,2,3,6,9,12, np.inf])


print(" Analisis de Cuando Arrancan los PL 2020 -2025")
pd.crosstab(kpis.qdate, kpis.r_mcc_med, margins = True).T

NameError: name 'pl' is not defined

* Deacuerdo a esta data existen puntos logisticos Nuevos **17** desde 2024 a la fecha

* solo **5** PL de alta demanda (tercer y cuarto cuartil) Tonelaje arriba de **304 Tons** promedio

In [207]:
pd.crosstab(index = kpis.r_mcc_med, columns = kpis.qdate , values = kpis.ton_media, aggfunc= "median").fillna(0).round(2)

C:\Users\adojeda\AppData\Local\Temp\ipykernel_23188\3618493216.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pd.crosstab(index = kpis.r_mcc_med, columns = kpis.qdate , values = kpis.ton_media, aggfunc= "median").fillna(0).round(2)


qdate,202000,202100,202200,202300,202401,202402,202403,202404,202501,202502
r_mcc_med,,,,,,,,,,
"(-0.158, 34.124]",2.40,2.56,3.37,7.92,0.00,19.16,2.10,23.81,20.03,0.00
"(34.124, 304.09]",130.07,93.22,190.99,168.35,0.00,0.00,88.04,0.00,0.00,147.51
"(304.09, 793.832]",593.09,487.87,547.71,555.92,402.86,0.00,0.00,0.00,0.00,437.30
"(793.832, 75848.546]",1296.75,3531.96,2380.36,4915.44,0.00,0.00,0.00,3305.46,1072.52,0.00


### Calculo de Frecuencias de Facturación Promedio 
* **1** es cada mes 
* **2** es cada dos meses
* **N** es cada **N** meses 


In [214]:
pd.crosstab( kpis.r_mcc_med, columns = kpis.r_freq,margins=True)

r_freq,"(0.0, 1.0]","(1.0, 2.0]","(2.0, 3.0]","(3.0, 6.0]","(6.0, 9.0]","(9.0, 12.0]","(12.0, inf]",All
r_mcc_med,,,,,,,,
"(-0.158, 34.124]",11,5,8,7,5,1,12,49
"(34.124, 304.09]",36,6,2,2,1,0,1,48
"(304.09, 793.832]",43,4,1,0,0,0,0,48
"(793.832, 75848.546]",46,2,0,0,0,0,0,48
All,136,17,11,9,6,1,13,193


existen **136** PL que tienen facturación mes a mes, **17** tienen facturación bimestral, **11** trimestral    

In [215]:
pd.crosstab( kpis.r_msc, columns = kpis.r_freq,margins=True)

r_freq,"(0.0, 1.0]","(1.0, 2.0]","(2.0, 3.0]","(3.0, 6.0]","(6.0, 9.0]","(9.0, 12.0]","(12.0, inf]",All
r_msc,,,,,,,,
"(-inf, 1.0]",95,5,2,0,1,1,2,106
"(1.0, 2.0]",12,5,1,2,0,0,1,21
"(2.0, 3.0]",3,1,3,0,1,0,1,9
"(3.0, 4.0]",1,1,0,0,0,0,1,3
"(4.0, 8.0]",6,2,1,3,1,0,2,15
"(8.0, 10.0]",1,0,0,2,0,0,0,3
"(10.0, 12.0]",5,0,1,1,1,0,0,8
"(12.0, inf]",13,3,3,1,2,0,6,28
All,136,17,11,9,6,1,13,193


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hashlib
from datetime import datetime
 
date = datetime.now().date()
m, y = date.month, date.year
min_date = date.replace(year= y-1, month = m, day = 1)
max_date = date.replace( month = m-1, day = 1) 
actual_date = date.replace(day = 1)
min_date, max_date, actual_date

(datetime.date(2024, 7, 1),
 datetime.date(2025, 6, 1),
 datetime.date(2025, 7, 1))

In [3]:
var= "toneladas_plan_ventas"
var2del = "toneladas_pvo"
pvo = pd.read_csv(r".\consultas\pvo.csv", engine="pyarrow")
pvo["Base"] = "pvo"
pvo = pvo.drop(columns= ["toneladas_dim_capacidades", var2del]) #"toneladas_pvo"
print(pvo.shape)
pvo = pvo.dropna(subset = [var, "nom_articulo", "nom_ubicacion"], how="any")
print(pvo.shape)
pvo["MesAct_fact"] = False

(1453564, 13)
(156458, 13)


In [4]:
fact = pd.read_csv(r".\consultas\fact.csv", engine="pyarrow")
fact["Base"] = "fact"
fact = fact[fact.MesAnio >= min_date]
fact["MesAct_fact"] = np.where( fact.MesAnio == actual_date, True, False)
fact.MesAnio.nunique()

13